# Group: Wen-Hsuan Hung, Hongyuan Zhang

# EDGAR - Reading Information Tables in Text Format - Advanced Text Mining (65 marks)

#### So far, we have collected CIKs for each of the Mutual Funds, then looked up the links of all the 13F HRs and the Information Tables, and identified them either text tables or xml tables. In class, we  obtained the information from the xml tables.  In this assignment we will obtain information from the text file.  These are not as nicely structured as the xml output.   In a csv (HW_Mutual_Fund_Info_Table_Link.csv) you will find a few of these links.  Your goal for this part of the homework is to obtain the link to the text files from the attached file  (HW_Mutual_Fund_Info_Table_Link.csv).  Then you will write a code that will go to the links of all the linked text files, and extract some columns.  Do not use Beutiful Soup for this assignment.   We provide some initial code to guide your initial steps:

#### First we have to collect the links of the text Info Tables in lists:

In [1]:
import urllib

text_link = []
text_Name = []
text_date = []

input_file = open('HW_Mutual_Fund_Info_Table_Link.csv', 'r')

rows = input_file.readlines()
input_file.close()

# Your Code on Enumerating the Lists.  The result should be three lists, text_link, text_Name,
#and text_date.  Each should have length 122.

for i in range(1, len(rows)):
    entrylist = rows[i].split(",")
    if entrylist[4] == "text":
        text_Name.append(entrylist[1])
        text_date.append(entrylist[2])
        text_link.append(entrylist[3])
print(len(text_Name))
print(len(text_date))
print(len(text_link)) 

FileNotFoundError: [Errno 2] No such file or directory: 'HW_Mutual_Fund_Info_Table_Link.csv'

#### Keep only the links that correspond to a date after 2010 (Don't inlude 2010, start at 2011).  Hint: you can use the datetime library.

In [2]:
from datetime import datetime

#Use the following list to store the filtered values.
filtered_dates = []
filtered_name = []
filtered_link = []

#Enter code here to to keep only the dates corresponding to after 2010.  

for i in range(1, len(rows)):
    entrylist = rows[i].split(",")
    x = datetime.strptime(entrylist[2], '%Y-%m-%d')
    if entrylist[4] == "text" and x.year > 2010:
        filtered_name.append(entrylist[1])
        filtered_dates.append(entrylist[2])
        filtered_link.append(entrylist[3])
print(len(filtered_name))
print(len(filtered_dates))
print(len(filtered_link))

NameError: name 'rows' is not defined

In [40]:
print(filtered_name)
print(filtered_dates)
print(filtered_link)

['Adirondack_Funds', 'ADVANCE_CAPITAL_I_INC', 'ADVANCE_CAPITAL_I_INC', 'ADVANCE_CAPITAL_I_INC', 'ADVANCE_CAPITAL_I_INC', 'ADVANCE_CAPITAL_I_INC', 'ADVANCE_CAPITAL_I_INC', 'Global_X_Funds']
['2013-05-06', '2013-05-01', '2013-02-08', '2012-11-07', '2012-08-09', '2012-05-15', '2012-02-02', '2011-02-14']
['/Archives/edgar/data/1311981/000116204413000513/0001162044-13-000513.txt', '/Archives/edgar/data/813470/000081347013000006/0000813470-13-000006.txt', '/Archives/edgar/data/813470/000081347013000001/0000813470-13-000001.txt', '/Archives/edgar/data/813470/000081347012000023/0000813470-12-000023.txt', '/Archives/edgar/data/813470/000081347012000019/0000813470-12-000019.txt', '/Archives/edgar/data/813470/000081347012000014/0000813470-12-000014.txt', '/Archives/edgar/data/813470/000081347012000003/0000813470-12-000003.txt', '/Archives/edgar/data/1432353/000114420411008428/0001144204-11-008428.txt']


#### Your filtered list should now have 8 elements.  These represent 3 mutual funds.   The first one has CIK=1311981 (from the link you can find this at data/1311981 ).  The second was has CIK 813470.  The third one has CIK 1432353. 

['https://www.sec.gov/Archives/edgar/data/1311981/000116204413000513/0001162044-13-000513.txt',
 'https://www.sec.gov/Archives/edgar/data/813470/000081347013000006/0000813470-13-000006.txt',
 'https://www.sec.gov/Archives/edgar/data/813470/000081347013000001/0000813470-13-000001.txt',
 'https://www.sec.gov/Archives/edgar/data/813470/000081347012000023/0000813470-12-000023.txt',
 'https://www.sec.gov/Archives/edgar/data/813470/000081347012000019/0000813470-12-000019.txt',
 'https://www.sec.gov/Archives/edgar/data/813470/000081347012000014/0000813470-12-000014.txt',
 'https://www.sec.gov/Archives/edgar/data/813470/000081347012000003/0000813470-12-000003.txt',
 'https://www.sec.gov/Archives/edgar/data/1432353/000114420411008428/0001144204-11-008428.txt']

#### Next, for each text link, extract the name of issuer, CUSIP, and the Quantity of shares.  You will also want to keep track of the mutual fund name as well as the filing report date.  

#### Your output file should have 5 columns.  The first is the issue date of the form which can be found in the filtered_date list (this will be repeated for the same form).  The second is the mutual fund name which can be found in the filtered_name list (this will be repeated).  The third, fourth and fifith are the name of issuer, CUSIP, and shares respectively.  Make sure to account for the fact that while some of the text files have the same formatting, others do not.  This means you will have to look through them to make sure your code works for the each text file. (Please make sure you use one chunk of code to process all the URLs, i.e. if we were to change the list of URLs your code should still work for those new URLs. Do not process the URLs separately.)

---

#### Since we will be loading a page for multiple URLs, sometimes the website can block us off since we will be requesting the many sites from the same IP address. We can bypass this issue by using something called as a headless browser. In the headless browser, we create a list of browsers and their header information which includes user agents, and then for every time we load a certain URL and read the HTML content, we use the code to randomly choose a browser header and user agent, thus fooling the server into believing that every time a different browser is knocking them, and so the server will be less likely to block us!

#### The following code has been provided to create this list of headers which you can randomly choose from later for each URL.

In [41]:
import requests
import random 
from collections import OrderedDict

headers_list = [
    # Firefox 77 Mac
    {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Referer": "https://www.google.com/",
    "DNT": "1",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1"
    },
    # Firefox 77 Windows
    {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate, br",
    "Referer": "https://www.google.com/",
    "DNT": "1",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1"
    },
    # Chrome 83 Mac
    {
    "Connection": "keep-alive",
    "DNT": "1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Sec-Fetch-Site": "none",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Dest": "document",
    "Referer": "https://www.google.com/",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8"
    },
    # Chrome 83 Windows 
    {
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-User": "?1",
    "Sec-Fetch-Dest": "document",
    "Referer": "https://www.google.com/",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9"
    }
]

Hint: If you wish to obtain the HTML content for a particular URL, you can use the following code:

```
    headers = random.choice(headers_list)
    r = requests.Session()
    r.headers = headers
    html = r.get(url).text
```



---

In [94]:
issue_date = []
mutual_fund_name = [] 
name_of_issuer = [] 
CUSIP = [] # CUSIP number
shares = [] # No. of Shares of the company in the Mutual Fund

# Your code goes here
repeat = []
number = 0

for url in filtered_link:
    link = "https://www.sec.gov" + url
    print(link)  

    headers = random.choice(headers_list)
    r = requests.Session()
    r.headers = headers

    html = r.get(link).text

    html = html.lower()

    start = html.find('name of issuer')
    end = html.find('</document>')
    html2 = html[start:end]
    html2 = html2.replace(" adr ", " adr  ")
    html2 = html2.replace(" $0.01 ", " $0.01   ")
    html2 = html2.replace(" $.001 ", " $0.01   ")
    html2 = html2.replace(" com com ", "  com com ")
    html2 = html2.replace(" $ ", "  ")
    html2 = html2.replace(" 892 ", " 892  ")
    html2 = html2.replace(" sole ", "  sole ")
    html2 = html2.replace("<s>", " ")
    html2 = html2.replace("<c>", " ")
    html2 = html2.replace("-", " ")
    html2 = html2.replace(" ma ", " ")
    html2 = html2.replace(" new ", " ")
    html2 = html2.replace(" cl ", " ")
    html2 = html2.replace("cl b ", " ")
    html2 = html2.replace(" rep ", " ")
    html2 = html2.replace(" pfd ", " ")
    html2 = html2.replace(" n v ", " ")
    html2 = html2.replace(" sek ", " ")
    html2 = html2.replace(" ser ", " ")
    html2 = html2.replace(" pref ", " ")

    start = html2.find("none\n")+5
    html2 = html2[start:]

    #print(html2.splitlines())

    html2 = html2.splitlines()

    for i in range(0, len(html2)):
        entrylist = html2[i].split("  ")
        try:
            while True:
                entrylist.remove('')
        except ValueError:
            pass

        if len(entrylist) > 4 and entrylist[0] != "name of issuer":
            name_of_issuer.append(entrylist[0])
            CUSIP.append(entrylist[2])
            shares.append(entrylist[4])  
            issue_date.append(filtered_dates[number])
            mutual_fund_name.append(filtered_name[number])
            
    print(len(name_of_issuer))
    print(len(CUSIP))
    print(len(shares))
    print(len(issue_date))
    print(len(mutual_fund_name))
    number += 1

https://www.sec.gov/Archives/edgar/data/1311981/000116204413000513/0001162044-13-000513.txt
96
96
96
96
96
https://www.sec.gov/Archives/edgar/data/813470/000081347013000006/0000813470-13-000006.txt
437
437
437
437
437
https://www.sec.gov/Archives/edgar/data/813470/000081347013000001/0000813470-13-000001.txt
789
789
789
789
789
https://www.sec.gov/Archives/edgar/data/813470/000081347012000023/0000813470-12-000023.txt
1129
1129
1129
1129
1129
https://www.sec.gov/Archives/edgar/data/813470/000081347012000019/0000813470-12-000019.txt
1469
1469
1469
1469
1469
https://www.sec.gov/Archives/edgar/data/813470/000081347012000014/0000813470-12-000014.txt
1828
1828
1828
1828
1828
https://www.sec.gov/Archives/edgar/data/813470/000081347012000003/0000813470-12-000003.txt
2169
2169
2169
2169
2169
https://www.sec.gov/Archives/edgar/data/1432353/000114420411008428/0001144204-11-008428.txt
2268
2268
2268
2268
2268


#### Finally, you write down the lists to an output CSV or Text file:

**Don't forget to submit your output files to Canvas as well!**

In [95]:
pwd

'/mnt/c/Users/Hongyuan Zhang/Desktop/MGTA414'

In [97]:
# Your code goes here

Output_File = open("Files_Directory/Edgar/A1PartA.txt", "w")

Output_File.write("name_of_issuer\t\t\t\tCUSIP\t\t\tshares\t\t\tissue_date\t\tmutual_fund_name\n")

result = ""
for x in range(0, len(name_of_issuer)):
    result += name_of_issuer[x].ljust(40) + CUSIP[x].ljust(25) +  shares[x].ljust(25) + issue_date[x].ljust(25) + mutual_fund_name[x].ljust(25) + '\t' + '\n'

Output_File.write(result)
Output_File.close()